In [1]:
# read ner.csv and generate triplet

In [77]:
import pandas as pd
df = pd.read_csv('./ner.csv')

In [10]:
df[:10]

,entity,label,index
0,I,B-MISC,3
1,F,B-MISC,3
2,I,B-MISC,3
3,La,B-MISC,10
4,##os,I-MISC,10
5,RF,B-ORG,13
6,##A,I-ORG,13
7,H,B-ORG,13
8,&,I-ORG,13
9,E,I-ORG,13


In [11]:
# rule 1 B-ORG|I-ORG locate B-LOC|I-LOC
# rule 2 B-PER|I-PER work B-ORG|I-ORG
# rule 3 B-PER|I-PER live B-LOC|I-LOC

In [78]:
dict_entity = {}
dict_label = {}
for i in range(len(df)):
    if df['index'][i] not in dict_entity.keys():
        dict_entity[df['index'][i]] = [df['entity'][i]]
        dict_label[df['index'][i]] = [df['label'][i]]
    else:
        dict_entity[df['index'][i]].append(df['entity'][i])
        dict_label[df['index'][i]].append(df['label'][i])

In [36]:
dict_label[13]

['B-ORG',
 'I-ORG',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'B-MISC',
 'B-ORG',
 'I-ORG',
 'B-MISC']

In [39]:
# rule 1
rule1Key = []
for key in dict_label.keys():
    if 'B-ORG' in dict_label[key] and 'B-LOC' in dict_label[key]:
        rule1Key.append(key)

In [43]:
dict_label[rule1Key[0]]
rule1Tuple = []


['B-LOC', 'B-LOC', 'I-LOC', 'B-ORG']

In [44]:
dict_entity[rule1Key[0]]

['Washington', 'Washington', 'State', 'E']

In [48]:
rule1Tuple = []
for key in rule1Key:
    tmpORG = ''
    tmpLoc = ''
    for i in range(len(dict_label[key])):
        if 'ORG' in dict_label[key][i]:
            tmpORG += dict_entity[key][i] + ' '
        if 'LOC' in dict_label[key][i]:
            tmpLoc += dict_entity[key][i] + ' '
    rule1Tuple.append((tmpORG, tmpLoc))

In [50]:
rule1Tuple[:10]

[('E ', 'Washington Washington State '),
 ('NHS ', 'UK UK '),
 ('Pro ##M Ha ##ptic ##a Ltd ', 'Ireland '),
 ('N ##E N ##E ', 'UK '),
 ('E ##MS ', 'UK '),
 ('##MS ', 'United States '),
 ('IS ##AC IS ##AC ', 'Mediterranean Area '),
 ('HC ##C ', 'Milan '),
 ('R ##EC Do ', 'China China '),
 ('GP ', 'Western Australia ')]

In [76]:
from py2neo import *
import re
graph = Graph("http://localhost:7474",user="neo4j",password="12345678",name="neo4j")

for i in range(len(rule1Tuple)):
    graph.run("merge (n:Organization { name:'" + re.sub('(\W+)', '', rule1Tuple[i][0]) + "'});")
    graph.run("merge (n:Location { name:'" + re.sub('(\W+)', '', rule1Tuple[i][1]) + "'});")
    graph.run("MATCH (a:Organization),(b:Location) WHERE a.name = '" + re.sub('(\W+)', '', rule1Tuple[i][0]) + "' AND b.name = '" + re.sub('(\W+)', '', rule1Tuple[i][1]) + "' merge (a)-[r:locate]->(b);")

In [79]:
# rule 2
rule2Key = []
for key in dict_label.keys():
    if 'B-PER' in dict_label[key] and 'B-ORG' in dict_label[key]:
        rule2Key.append(key)

rule2Tuple = []
for key in rule2Key:
    tmpORG = ''
    tmpLoc = ''
    for i in range(len(dict_label[key])):
        if 'PER' in dict_label[key][i]:
            tmpORG += dict_entity[key][i] + ' '
        if 'ORG' in dict_label[key][i]:
            tmpLoc += dict_entity[key][i] + ' '
    rule2Tuple.append((tmpORG, tmpLoc))


for i in range(len(rule2Tuple)):
    graph.run("merge (n:Person { name:'" + re.sub('(\W+)', '', rule2Tuple[i][0]) + "'});")
    graph.run("merge (n:Organization { name:'" + re.sub('(\W+)', '', rule2Tuple[i][1]) + "'});")
    graph.run("MATCH (a:Person),(b:Organization) WHERE a.name = '" + re.sub('(\W+)', '', rule2Tuple[i][0]) + "' AND b.name = '" + re.sub('(\W+)', '', rule2Tuple[i][1]) + "' merge (a)-[r:work]->(b);")

In [80]:
# rule 3
# rule 3 B-PER|I-PER live B-LOC|I-LOC
rule3Key = []
for key in dict_label.keys():
    if 'B-PER' in dict_label[key] and 'B-LOC' in dict_label[key]:
        rule3Key.append(key)

rule3Tuple = []
for key in rule3Key:
    tmpORG = ''
    tmpLoc = ''
    for i in range(len(dict_label[key])):
        if 'PER' in dict_label[key][i]:
            tmpORG += dict_entity[key][i] + ' '
        if 'LOC' in dict_label[key][i]:
            tmpLoc += dict_entity[key][i] + ' '
    rule3Tuple.append((tmpORG, tmpLoc))


for i in range(len(rule3Tuple)):
    graph.run("merge (n:Person { name:'" + re.sub('(\W+)', '', rule3Tuple[i][0]) + "'});")
    graph.run("merge (n:Location { name:'" + re.sub('(\W+)', '', rule3Tuple[i][1]) + "'});")
    graph.run("MATCH (a:Person),(b:Location) WHERE a.name = '" + re.sub('(\W+)', '', rule3Tuple[i][0]) + "' AND b.name = '" + re.sub('(\W+)', '', rule3Tuple[i][1]) + "' merge (a)-[r:live]->(b);")

In [2]:
from py2neo import *
import re
graph = Graph("http://localhost:7474",user="neo4j",password="12345678",name="neo4j")

graph.run("merge (n:Person { name:'SanPedroSula'});")
graph.run("MATCH (a:Person),(b:Location) WHERE a.name = 'SanPedroSula' AND b.name = 'UnitedStates' merge (a)-[r:live]->(b);")


(No data)

In [5]:
graph.run("merge (n:Person { name:'Ester'});")

(No data)

In [6]:
graph.run("MATCH (a:Person),(b:Location) WHERE a.name = 'Ester' AND b.name = 'California' merge (a)-[r:live]->(b);")


(No data)